### Hello All, I am a student currently learning about GAN. I am using this notebook to learn and remember about GAN, you can use this too for your benefit. Iam reffering to the yt video - https://www.youtube.com/watch?v=OljTVUVzPpM&t=189s&ab_channel=AladdinPersson to learn GAN code. ( I am a learner so might have some mistakes, please let me know so i can correct them and learn better)

## GAN Formula
1. GAN consists of mainly discriminators and generators (just names) where each of them are seperate AI models built to correct each other.
2. GAN is generally used to generate new information in the form of images or texts. That is why the name, Genrative Adversial Network.
3. The job of the genrator model is to generate new information with its best accuracy while the job of thw discriminator is to classify the predicted data as a fake or a real when passed through it while comparing to real training data.
4. The optimizer for the generator improves the generator to produce better information or generate better information whereas the job for the optimizer of discriminator is to improve the predictability of the discriminator to classify between the fake and real data.
5. Discriminator mainly works on the equation- `log(D(real)) + log(1-D(G(noise)))` where the discriminator tries to max out this equation . So, it tries to reduce the noise and improve the real collected data.
6. Generator works on the equation- `log(D(G(noise)))`. Its goal is to minimize this function to produce the best image. If noise is reduced, the model generates better data.

##Let us see the working on MNIST dataset

##Importing all necessary libraries and packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter


In [9]:
#Building Discriminator Model

class Discriminator(nn.Module):
  def __init__(self,img_dim):
    super().__init__()
    self.gen=nn.Sequential(
        nn.Linear(img_dim,256),
        nn.LeakyReLU(0.1),
        nn.Linear(256,1),
        nn.Sigmoid()
    )
  def forward(self,x):
    return self.gen(x)

In [10]:
#Building Generator Model

class Generator(nn.Module):
  def __init__(self,img_dim,noise_dim):
    super().__init__()
    self.gen=nn.Sequential(
        nn.Linear(noise_dim,256),
        nn.LeakyReLU(0.1),
        nn.Linear(256,img_dim),
        nn.Tanh() #using tanh as it gives the output as a value btw -1 and 1 and we are going to normalise the dataset between -1 and 1.
    )
  def forward(self,x):
    return self.gen(x)

In [4]:
#hyperparameters (incredbily sensitive to hyperparamters)

device='cuda' if torch.cuda.is_available() else 'cpu'
lr=3e-4 #allegedly the best value for learning rate of adam optimizer
noise_dim=64 #random
img_dim=28*28*1 #flat value
batch_size=32
num_epochs=50

In [11]:
disc=Discriminator(img_dim).to(device)
gen=Generator(img_dim,noise_dim).to(device)
fixed_noise=torch.randn((batch_size,noise_dim)).to(device) #torch.randn creates random number for the given tensor size witht the mean of 0 and standard deviation of 1
transforms=transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))] #totensor changes the numpy array conversion of the data to a tensor and normalize normalizes the value between wahtever specification is given

)
dataset=datasets.MNIST(root="dataset/",transform=transforms,download=True)
loader=DataLoader(dataset,batch_size=batch_size,shuffle=True)
opt_disc=optim.Adam(disc.parameters(),lr=lr)
opt_gen=optim.Adam(gen.parameters(),lr=lr)
criterion=nn.BCELoss() #loss function
writer_fake=SummaryWriter(f"runs/GAN_MNIST/fake") #stores all generated images
writer_real=SummaryWriter(f"runs/GAN_MNIST/real") #has all real images
step=0

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:07<00:00, 1389121.34it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 125006.52it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:03<00:00, 512035.87it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3224530.94it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [13]:
#training loop

for epoch in range(num_epochs):
  for batch_idx, (real,_) in enumerate(loader): #not taking labels means data is unsupervised
    real=real.view(-1,784).to(device)
    batch_size=real.shape[0]

    #training discriminator (maximising (`log(D(real)+log(1-D(G(noise)))`))
    noise=torch.randn(batch_size,noise_dim).to(device)
    fake=gen(noise)
    disc_real=disc(real).view(-1)
    lossD_real=criterion(disc_real,torch.ones_like(disc_real)) #maximising d(real) #close to 1
    disc_fake=disc(fake).view(-1)
    lossD_fake=criterion(disc_fake,torch.zeros_like(disc_fake)) #minimizing d(g(noise)) #close to 0
    lossD=(lossD_real+lossD_fake)/2
    disc.zero_grad()
    lossD.backward(retain_graph=True)#using retain graph helps us to retain the loss value whichwould hve gotten removed after using the backward function t save memory
    opt_disc.step()

    #training generator (minimizing (`log(1-D(G(noise)))` or max of `D(G(noise))` (to prevent the model to stop optimizing)))
    output=disc(fake).view(-1)
    lossG=criterion(output,torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    if batch_idx==0:
      print(
          f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
          Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
      )

      with torch.no_grad():
        fake=gen(fixed_noise).reshape(-1,1,28,28)
        data=real.reshape(-1,1,28,28)
        img_grid_fake=torchvision.utils.make_grid(fake,normalize=True)
        img_grid_real=torchvision.utils.make_grid(data,normalize=True)
        writer_fake.add_image("MNIST fake images",img_grid_fake,global_step=step)
        writer_real.add_image("MNIST real images",img_grid_real,global_step=step)
        step+=1



Epoch [0/50] Batch 0/1875           Loss D: 0.5418, loss G: 0.6971
Epoch [1/50] Batch 0/1875           Loss D: 0.4653, loss G: 1.1987
Epoch [2/50] Batch 0/1875           Loss D: 0.4055, loss G: 1.2207
Epoch [3/50] Batch 0/1875           Loss D: 0.3936, loss G: 1.3665
Epoch [4/50] Batch 0/1875           Loss D: 0.4746, loss G: 1.3271
Epoch [5/50] Batch 0/1875           Loss D: 0.5546, loss G: 1.0204
Epoch [6/50] Batch 0/1875           Loss D: 0.4737, loss G: 1.6329
Epoch [7/50] Batch 0/1875           Loss D: 0.5120, loss G: 1.3777
Epoch [8/50] Batch 0/1875           Loss D: 0.5833, loss G: 1.4158
Epoch [9/50] Batch 0/1875           Loss D: 0.7977, loss G: 1.0708
Epoch [10/50] Batch 0/1875           Loss D: 0.5976, loss G: 1.0690
Epoch [11/50] Batch 0/1875           Loss D: 0.6568, loss G: 1.2099
Epoch [12/50] Batch 0/1875           Loss D: 0.4803, loss G: 1.4783
Epoch [13/50] Batch 0/1875           Loss D: 0.5705, loss G: 1.3571
Epoch [14/50] Batch 0/1875           Loss D: 0.5575, loss 